In [1]:
# Retrieval Augmented Generation using Azure documentation
from dotenv import dotenv_values
from langchain_openai import ChatOpenAI


config = dotenv_values(".env")

llm = ChatOpenAI(api_key=config["OPEN_AI_KEY"])

from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://blog.suny.edu/36201/10-reasons-why-an-internship-is-important-to-all-students/")

docs = loader.load()

In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

embeddings = OpenAIEmbeddings(api_key=config["OPEN_AI_KEY"])

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [3]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [4]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [6]:
response = retrieval_chain.invoke({"input": "Of the 10 reasons discussed on why you need an internship, pick 3 and discuss in 2-3 sentences each why those reasons are the most important to you in your internship"})
print(response["answer"])


1. Application of education and career exploration: This reason is important to me because it allows me to connect what I have learned in the classroom to real-world experiences, helping me explore different career paths and specializations that align with my interests.

2. Networking: Networking is crucial as it helps me build relationships with professionals in my desired field, increasing my chances of securing a job in the future. It also provides me with valuable connections and opportunities for growth and development.

3. Gain experience and increase marketability: Having an internship gives me hands-on experience in my chosen career field, making me more competitive in the job market. It not only prepares me for what to expect in my field but also boosts my confidence in my abilities and work.
